REMARKS:
- Same flow as notebook 002-convert-spacy-to-iob2 but instead of sample OpenAI data this notebook processes all data outputed from OSS LLM Llama3

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

In [3]:
import sys
sys.path.insert(0, '..')

from loguru import logger

# Constants

In [4]:
VERSION = '20240706095040'

# Load data

In [5]:
# llm output
llm_output_spacy_fp = f"../data/output/llm_extract_output_{VERSION}_spacy.json"
with open(llm_output_spacy_fp, "r") as f:
    llm_output = json.load(f)

In [6]:
# holdout
holdout_spacy_fp = "../data/output/llm_extract_output_holdout_spacy.json"
with open(holdout_spacy_fp, "r") as f:
    holdout = json.load(f)

# Exclude holdout from llm output

In [7]:
holdout

[{'id': '53745975096161334446695170010345366844',
  'text': 'The restaurant looks out over beautiful green lawns to the Hudson River and the Statue of Liberty.',
  'label': [[15, 51, 'VIEW'], [59, 97, 'VIEW']],
  'Comments': []},
 {'id': '14008356808251230170256576349676573112',
  'text': "Also, the sandwiches (nearing $7) didn't come with anything like chips or a side.",
  'label': [[10, 33, 'PRICE'], [10, 80, 'FOOD']],
  'Comments': []},
 {'id': '196846222781357184059657962635349503302',
  'text': 'Luckily we saved room for the BBQ Salmon, Sea Bass and Crispy Duck.',
  'label': [[30, 66, 'FOOD']],
  'Comments': []},
 {'id': '122902927917699489714278789756481778575',
  'text': "I've been to Naples 45 for dinner twice.",
  'label': [[0, 39, 'SERVICE']],
  'Comments': []},
 {'id': '209028716300947491053334478581226250868',
  'text': 'Haru serves very fresh fish, has a trendy, modern ambiance, prime location on Park Avenue South and friendly service.',
  'label': [[5, 27, 'FOOD'],
   [35

In [8]:
holdout_texts = set([e['text'] for e in holdout])
llm_output_exc = [e for e in llm_output if e['text'] not in holdout_texts]
assert len(llm_output_exc) == (len(llm_output) - len(holdout))

# Convert to IOB2

In [9]:
from src.convert_ner_format.spacy_to_iob2 import convert_from_spacy_to_iob2

In [10]:
holdout_iob2 = convert_from_spacy_to_iob2(holdout)
llm_output_exc_iob2 = convert_from_spacy_to_iob2(llm_output_exc)

2024-07-09 10:58:17.226 | INFO     | src.utils.time.timer:timed:23 - convert_from_spacy_to_iob2 runtime: 0.043s
2024-07-09 10:58:23.701 | INFO     | src.utils.time.timer:timed:23 - convert_from_spacy_to_iob2 runtime: 6.474s


In [11]:
logger.info(f"{len(holdout_iob2)=}, {len(llm_output_exc_iob2)=}")

2024-07-09 10:58:23.716 | INFO     | __main__:<module>:1 - len(holdout_iob2)=10, len(llm_output_exc_iob2)=1988


# Add metadata

In [12]:
from src.convert_ner_format.spacy_to_iob2 import add_metadata

In [13]:
holdout_iob2_conll = add_metadata(holdout_iob2, holdout)
llm_output_exc_iob2_conll = add_metadata(llm_output_exc_iob2, llm_output_exc)

# Convert Tags to Int

In [14]:
from src.convert_ner_format.spacy_to_iob2 import build_ner_tags_label
from pprint import pprint

In [15]:
ner_tags_label = build_ner_tags_label(llm_output_exc_iob2_conll)
ner_tags_label_mapper = {i: v for i, v in enumerate(ner_tags_label)}
ner_tags_label_reverse_mapper = {v: i for i, v in enumerate(ner_tags_label)}

print("ner_tags_label_mapper:")
pprint(ner_tags_label_mapper)

print("ner_tags_label_reverse_mapper:")
pprint(ner_tags_label_reverse_mapper)

ner_tags_label_mapper:
{0: 'O',
 1: 'B-AMBIENCE',
 2: 'I-AMBIENCE',
 3: 'B-BEVERAGE',
 4: 'I-BEVERAGE',
 5: 'B-FOOD',
 6: 'I-FOOD',
 7: 'B-LOCATION',
 8: 'I-LOCATION',
 9: 'B-OVERALL',
 10: 'I-OVERALL',
 11: 'B-PRICE',
 12: 'I-PRICE',
 13: 'B-SERVICE',
 14: 'I-SERVICE',
 15: 'B-STAFF',
 16: 'I-STAFF',
 17: 'B-VALUE',
 18: 'I-VALUE',
 19: 'B-VIEW',
 20: 'I-VIEW'}
ner_tags_label_reverse_mapper:
{'B-AMBIENCE': 1,
 'B-BEVERAGE': 3,
 'B-FOOD': 5,
 'B-LOCATION': 7,
 'B-OVERALL': 9,
 'B-PRICE': 11,
 'B-SERVICE': 13,
 'B-STAFF': 15,
 'B-VALUE': 17,
 'B-VIEW': 19,
 'I-AMBIENCE': 2,
 'I-BEVERAGE': 4,
 'I-FOOD': 6,
 'I-LOCATION': 8,
 'I-OVERALL': 10,
 'I-PRICE': 12,
 'I-SERVICE': 14,
 'I-STAFF': 16,
 'I-VALUE': 18,
 'I-VIEW': 20,
 'O': 0}


In [17]:
from src.convert_ner_format.spacy_to_iob2 import convert_tags_to_int

In [19]:
llm_output_int_tag = convert_tags_to_int(llm_output_exc_iob2_conll, ner_tags_label_reverse_mapper)
holdout_int_tag = convert_tags_to_int(holdout_iob2_conll, ner_tags_label_reverse_mapper)

#### Persist

In [20]:
# Split to train and validation
from sklearn.model_selection import train_test_split

train_outputs, val_outputs = train_test_split(llm_output_int_tag, test_size=0.2, random_state=42)

print(f"{len(train_outputs)=}, {len(val_outputs)=}")

len(train_outputs)=1590, len(val_outputs)=398


In [21]:
def save_to_jsonl(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            json_line = json.dumps(item)
            f.write(json_line + '\n')

In [22]:
iob2_formatted_persist_path = '../data/llama3_extracted_iob2_format_{split}.jsonl'
iob2_train = train_outputs[:]
iob2_val = val_outputs[:]
iob2_test = holdout_int_tag[:]

save_to_jsonl(iob2_train, iob2_formatted_persist_path.format(split='train'))
save_to_jsonl(iob2_val, iob2_formatted_persist_path.format(split='val'))
save_to_jsonl(iob2_test, iob2_formatted_persist_path.format(split='test'))

# Push dataset to HuggingFace Datasets

## Add Dataset features

In [23]:
from datasets import Features, ClassLabel, Sequence, Value

In [24]:
ner_feature = Sequence(feature=ClassLabel(num_classes=len(ner_tags_label), names=ner_tags_label, id=None))

features = Features(
    {
        'id': Value('string'),
        'text': Value('large_string'),
        'Comments': Sequence(Value('string')),
        'tokens': Sequence(Value('string')),
        'ner_tags': ner_feature
    }
)

In [25]:
from datasets import load_dataset, Dataset

data_files = {
    'train': iob2_formatted_persist_path.format(split='train'),
    'val': iob2_formatted_persist_path.format(split='val'),
    'test': iob2_formatted_persist_path.format(split='test'),
}
dataset = load_dataset('json', data_files=data_files, features=features)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [26]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'Comments', 'tokens', 'ner_tags'],
        num_rows: 1590
    })
    val: Dataset({
        features: ['id', 'text', 'Comments', 'tokens', 'ner_tags'],
        num_rows: 398
    })
    test: Dataset({
        features: ['id', 'text', 'Comments', 'tokens', 'ner_tags'],
        num_rows: 10
    })
})

## Push

In [27]:
from dotenv import load_dotenv

load_dotenv()

True

In [28]:
!huggingface-cli login --token $HUGGINGFACE_WRITE_TOKEN --add-to-git-credential

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/dvquys/.cache/huggingface/token
Login successful


In [29]:
HUGGINGFACE_DS = 'dvquys/restaurant-reviews-public-sources'
dataset.push_to_hub(HUGGINGFACE_DS, commit_message='Register Features')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/800 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/dvquys/restaurant-reviews-public-sources/commit/85e79ec45713b81c0938397f961523de1099097d', commit_message='Register Features', commit_description='', oid='85e79ec45713b81c0938397f961523de1099097d', pr_url=None, pr_revision=None, pr_num=None)